# Code Generator

The requirement: use a Frontier model to generate high performance C++ code from Python code


<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Reminder: fetch latest code</h2>
            <span style="color:#f71;">I'm continually improving these labs, adding more examples and exercises.
            At the start of each week, it's worth checking you have the latest code.<br/>
            First do a <a href="https://chatgpt.com/share/6734e705-3270-8012-a074-421661af6ba9">git pull and merge your changes as needed</a>. Any problems? Try asking ChatGPT to clarify how to merge - or contact me!<br/><br/>
            After you've pulled the code, from the llm_engineering directory, in a Cursor Terminal, run:<br/>
            <code>uv sync</code><br/>
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h1 style="color:#900;">Important Note</h1>
            <span style="color:#900;">
            In this lab, I use high end models GPT 5, Claude 4.5 Sonnet, Gemini 2.5 Pro, Grok 4, which are the slightly higher priced models. The costs are still low, but if you'd prefer to keep costs ultra low, please pick lower cost models like gpt-5-nano.
            </span>
        </td>
    </tr>
</table>

In [1]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import subprocess
from IPython.display import Markdown, display

In [9]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
grok_api_key = os.getenv('GROK_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set (and this is optional)")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if grok_api_key:
    print(f"Grok API Key exists and begins {grok_api_key[:4]}")
else:
    print("Grok API Key not set (and this is optional)")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AI
Grok API Key not set (and this is optional)


In [3]:
# Connect to client libraries

openai = OpenAI()

anthropic_url = "https://api.anthropic.com/v1/"
gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
grok_url = "https://api.x.ai/v1"

anthropic = OpenAI(api_key=anthropic_api_key, base_url=anthropic_url)
gemini = OpenAI(api_key=google_api_key, base_url=gemini_url)
grok = OpenAI(api_key=grok_api_key, base_url=grok_url)

In [4]:
OPENAI_MODEL = "gpt-5"
CLAUDE_MODEL = "claude-sonnet-4-5-20250929"
# GROK_MODEL = "grok-4"
GEMINI_MODEL = "gemini-2.5-pro"

# Want to keep costs ultra-low? Uncomment these lines:

# OPENAI_MODEL = "gpt-5-nano"
# CLAUDE_MODEL = "claude-3-5-haiku-latest"
# GROK_MODEL = "grok-4-fast-non-reasoning"
# GEMINI_MODEL = "gemini-2.5-flash-lite"

## PLEASE NOTE:

We will be writing a solution to convert Python into efficient, optimized C++ code for your machine, which can be compiled to native machine code and executed.

It is not necessary for you to execute the code yourself - that's not the point of the exercise!

But if you would like to (because it's satisfying!) then I'm including the steps here. Very optional!

As an alternative, I'll also show you a website where you can run the C++ code.

In [5]:
from system_info import retrieve_system_info

system_info = retrieve_system_info()
system_info

{'os': {'system': 'Darwin',
  'arch': 'arm64',
  'release': '24.6.0',
  'version': 'Darwin Kernel Version 24.6.0: Mon Aug 11 21:15:09 PDT 2025; root:xnu-11417.140.69.701.11~1/RELEASE_ARM64_T6041',
  'kernel': '24.6.0',
  'distro': None,
  'wsl': False,
  'rosetta2_translated': False,
  'target_triple': 'arm64-apple-darwin24.6.0'},
 'package_managers': ['xcode-select (CLT)', 'brew'],
 'cpu': {'brand': 'Apple M4 Pro',
  'cores_logical': 14,
  'cores_physical': 14,
  'simd': []},
 'toolchain': {'compilers': {'gcc': 'Apple clang version 17.0.0 (clang-1700.3.19.1)',
   'g++': 'Apple clang version 17.0.0 (clang-1700.3.19.1)',
   'clang': 'Apple clang version 17.0.0 (clang-1700.3.19.1)',
   'msvc_cl': ''},
  'build_tools': {'cmake': 'cmake version 4.1.2',
   'ninja': '',
   'make': 'GNU Make 3.81'},
  'linkers': {'ld_lld': ''}}}

In [12]:
message = f"""
Here is a report of the system information for my computer.
I want to run a C++ compiler to compile a single C++ file called main.cpp and then execute it in the simplest way possible.
Please reply with whether I need to install any C++ compiler to do this. If so, please provide the simplest step by step instructions to do so.

If I'm already set up to compile C++ code, then I'd like to run something like this in Python to compile and execute the code:
```python
compile_command = # something here - to achieve the fastest possible runtime performance
compile_result = subprocess.run(compile_command, check=True, text=True, capture_output=True)
run_command = # something here
run_result = subprocess.run(run_command, check=True, text=True, capture_output=True)
return run_result.stdout
```
Please tell me exactly what I should use for the compile_command and run_command.

System information:
{system_info}
"""

response = gemini.chat.completions.create(model=GEMINI_MODEL, messages=[{"role": "user", "content": message}])
display(Markdown(response.choices[0].message.content))
    

Excellent, you have provided very clear system information. Let's get straight to the answers.

### Do you need to install a C++ compiler?

No, you do not need to install a C++ compiler.

Your system report clearly shows that you are already set up. Specifically, this section indicates you have the necessary tools installed via Apple's Xcode Command Line Tools:

```
'toolchain': {
    'compilers': {
        'g++': 'Apple clang version 17.0.0 (clang-1700.3.19.1)',
        'clang': 'Apple clang version 17.0.0 (clang-1700.3.19.1)'
    }
}
```

On macOS, `g++` is an alias for `clang++`, which is Apple's excellent, modern C++ compiler. You are ready to go.

---

### Python Script to Compile and Run

Here is the Python code with the `compile_command` and `run_command` variables filled in to compile your `main.cpp` for the best possible runtime performance and then execute it.

```python
import subprocess
import os

# Ensure there is a main.cpp file to compile for this example to work.
# For example, create a file named main.cpp with the following content:
# #include <iostream>
# int main() {
#     std::cout << "Hello from C++ on Apple M4 Pro!" << std::endl;
#     return 0;
# }

def compile_and_run_cpp():
    """
    Compiles main.cpp for maximum performance and runs the resulting executable.
    """
    source_file = "main.cpp"
    executable_name = "main_executable"

    # For the fastest possible runtime performance, we use high optimization levels.
    # On macOS, g++ is an alias for clang++, so we can use either.
    compile_command = [
        "clang++",
        "-std=c++17",      # Use a modern C++ standard
        "-O3",             # The highest standard level of optimization for speed
        "-march=native",   # Optimize specifically for your M4 Pro CPU
        "-DNDEBUG",        # Disables asserts and other debug code
        "-o", executable_name, # Specify the output file name
        source_file        # The input C++ source file
    ]

    run_command = [f"./{executable_name}"]

    try:
        print(f"Compiling with command: {' '.join(compile_command)}")
        # Compile the C++ code
        compile_result = subprocess.run(
            compile_command, 
            check=True,          # Raises an exception if compilation fails
            text=True, 
            capture_output=True
        )
        if compile_result.stderr:
            print("Compiler Warnings/Messages:\n", compile_result.stderr)
        
        print(f"\nRunning with command: {' '.join(run_command)}")
        # Run the compiled executable
        run_result = subprocess.run(
            run_command, 
            check=True, 
            text=True, 
            capture_output=True
        )

        # Clean up the created executable
        os.remove(executable_name)

        return run_result.stdout

    except FileNotFoundError:
        return f"Error: Command not found. Is '{compile_command[0]}' in your PATH?"
    except subprocess.CalledProcessError as e:
        # This will catch errors from both compilation and execution
        error_message = (
            f"An error occurred.\n"
            f"Command: {' '.join(e.cmd)}\n"
            f"Return Code: {e.returncode}\n"
            f"Stdout:\n{e.stdout}\n"
            f"Stderr:\n{e.stderr}"
        )
        return error_message

# --- To run this code ---
# 1. Save the python script (e.g., as `runner.py`).
# 2. Create a file named `main.cpp` in the same directory.
# 3. Run the python script from your terminal: `python3 runner.py`

if __name__ == '__main__':
    output = compile_and_run_cpp()
    print("\n--- C++ Program Output ---")
    print(output)
    print("--------------------------")
```

### Explanation of the Commands

#### `compile_command`
`['clang++', '-std=c++17', '-O3', '-march=native', '-DNDEBUG', '-o', 'main_executable', 'main.cpp']`

*   `clang++`: The C++ compiler on your system.
*   `-std=c++17`: Specifies that the code should be compiled using the C++17 standard. This is a good, modern default. You could also use `-std=c++20`.
*   `-O3`: This is the key flag for performance. It tells the compiler to apply a high level of aggressive optimizations to make the code run as fast as possible.
*   `-march=native`: This is another important performance flag. It instructs the compiler to generate code specifically optimized for the CPU you are compiling on (in your case, the Apple M4 Pro). This can unlock specific instruction sets for better performance.
*   `-DNDEBUG`: This preprocessor directive is standard practice for "release" or "production" builds. It typically disables `assert()` statements and other debugging-only code, which can slightly improve performance and reduce binary size.
*   `-o main_executable`: This sets the name of the output file (the compiled program) to `main_executable`. If you omit this, it will default to `a.out`.
*   `main.cpp`: This is your input source file.

#### `run_command`
`['./main_executable']`

*   `./main_executable`: This command tells the shell to execute the file named `main_executable` located in the current directory (`.`).

## If you need to install something

If you would like to, please follow GPTs instructions! Then rerun the analysis afterwards (you might need to Restart the notebook) to confirm you're set.

You should now be equipped with the command to compile the code, and the command to run it!

Enter that in the cell below:

In [13]:
compile_command = ["clang++", "-std=c++17", "-Ofast", "-mcpu=native", "-flto=thin", "-fvisibility=hidden", "-DNDEBUG", "main.cpp", "-o", "main"]
run_command = ["./main"]

## And now, on with the main task

In [14]:
system_prompt = """
Your task is to convert Python code into high performance C++ code.
Respond only with C++ code. Do not provide any explanation other than occasional comments.
The C++ response needs to produce an identical output in the fastest possible time.
"""

def user_prompt_for(python):
    return f"""
Port this Python code to C++ with the fastest possible implementation that produces identical output in the least time.
The system information is:
{system_info}
Your response will be written to a file called main.cpp and then compiled and executed; the compilation command is:
{compile_command}
Respond only with C++ code.
Python code to port:

```python
{python}
```
"""

In [15]:
def messages_for(python):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(python)}
    ]
 

In [16]:
def write_output(cpp):
    with open("main.cpp", "w", encoding="utf-8") as f:
        f.write(cpp)

In [17]:
def port(client, model, python):
    reasoning_effort = "high" if 'gpt' in model else None
    response = client.chat.completions.create(model=model, messages=messages_for(python), reasoning_effort=reasoning_effort)
    reply = response.choices[0].message.content
    reply = reply.replace('```cpp','').replace('```','')
    write_output(reply)

In [21]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(400_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [19]:
def run_python(code):
    globals = {"__builtins__": __builtins__}
    exec(code, globals)

In [22]:
run_python(pi)

Result: 3.141592654838
Execution Time: 21.911236 seconds


In [23]:
port(anthropic, CLAUDE_MODEL, pi)

# Compiling C++ and executing

This next cell contains the command to compile a C++ file based on the instructions from GPT.

Again, it's not crucial to do this step if you don't wish to!

OR alternatively: student Sandeep K.G. points out that you can run Python and C++ code online to test it out that way. Thank you Sandeep!  
> Not an exact comparison but you can still get the idea of performance difference.  
> For example here: https://www.programiz.com/cpp-programming/online-compiler/

In [24]:
# Use the commands from GPT 5

def compile_and_run():
    subprocess.run(compile_command, check=True, text=True, capture_output=True)
    print(subprocess.run(run_command, check=True, text=True, capture_output=True).stdout)
    print(subprocess.run(run_command, check=True, text=True, capture_output=True).stdout)
    print(subprocess.run(run_command, check=True, text=True, capture_output=True).stdout)

In [25]:
compile_and_run()

Result: 3.141592654839
Execution Time: 0.162977 seconds

Result: 3.141592654839
Execution Time: 0.150832 seconds

Result: 3.141592654839
Execution Time: 0.153744 seconds



In [26]:
21.911236/0.162977

134.44373132405184

## OK let's try the other contenders!

In [ ]:
port(anthropic, CLAUDE_MODEL, pi)
compile_and_run()

In [ ]:
port(grok, GROK_MODEL, pi)
compile_and_run()

In [27]:
port(gemini, GEMINI_MODEL, pi)
compile_and_run()


Result: 3.141592654838
Execution Time: 0.025465 seconds

Result: 3.141592654838
Execution Time: 0.022107 seconds

Result: 3.141592654838
Execution Time: 0.022870 seconds



In [ ]:
print(f"""
In Ed's experiments, the performance speedups were:

4th place: Claude Sonnet 4.5: {19.178207/0.104241:.0f}X speedup
3rd place: GPT-5: {19.178207/0.082168:.0f}X speedup
2nd place: Grok 4: {19.178207/0.018092:.0f}X speedup
1st place: Gemini 2.5 Pro: {19.178207/0.013314:.0f}X speedup
""")